<a href="https://colab.research.google.com/github/cmhrabi/AgenticDesignPatterns/blob/main/notebooks/Chapter_3_Parallelization_(LangChain_Code_Example).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade langchain langchain-openai langchain-core pydantic

     |████████████████████████████████| 1.0 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 27.2 MB/s eta 0:00:01
     |████████████████████████████████| 449 kB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 94.4 MB/s eta 0:00:01
  Using cached requests-2.32.5-py3-none-any.whl (64 kB)
     |████████████████████████████████| 396 kB 105.0 MB/s eta 0:00:01
     |████████████████████████████████| 174 kB 43.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 59.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 53.2 MB/s eta 0:00:01
     |████████████████████████████████| 997 kB 26.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 103.5 MB/s eta 0:00:01
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
     |████████████████████████████████| 238 kB 51.8 MB/s eta 0:00:01
     |████████████████████████████████| 640 kB 10.3 MB/s eta 0:00:01
  Using cached httpx-0.28.1-py3-none-any.whl (73

In [12]:
import os, getpass 
import asyncio 
import nest_asyncio 

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.tools import tool as langchain_tool 
from langchain.agents import create_tool_calling_agent, AgentExecutor

# --- Configuration ---
# Ensure your API key environment variable is set (e.g., OPENAI_API_KEY)
try:
    llm = ChatOpenAI(
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_BASE_URL"),
        model="anthropic/claude-sonnet-4",
        temperature=0
    )
    if llm:
        print(f"Language model initialized: {llm.model_name}")
except Exception as e:
    print(f"Error initializing language model: {e}")
    llm = None

# --- Define a Tool --- 
@langchain_tool 
def search_information(query: str) -> str: 
    """ 
    Provides factual information on a given topic. Use this tool to 
    find answers to phrases 
    like 'capital of France' or 'weather in London?'. 
    """ 
    print(f"\n--- 🛠 Tool Called: search_information with query: '{query}' ---") 
    # Simulate a search tool with a dictionary of predefined results. 
    simulated_results = { 
        "weather in london": "The weather in London is currently cloudy with a temperature of 15°C.", 
        "capital of france": "The capital of France is Paris.", 
        "population of earth": "The estimated population of Earth is around 8 billion people.", 
        "tallest mountain": "Mount Everest is the tallest mountain above sea level.", 
        "default": f"Simulated search result for '{query}': No specific information found, but the topic seems interesting." 
    } 
    result = simulated_results.get(query.lower(), 
    simulated_results["default"]) 
    print(f"--- TOOL RESULT: {result} ---") 
    return result 
 
tools = [search_information] 
 
# --- Create a Tool-Calling Agent --- 
if llm: 
    # This prompt template requires an `agent_scratchpad` placeholder for the agent's internal steps. 
    agent_prompt = ChatPromptTemplate.from_messages([ 
        ("system", "You are a helpful assistant."), 
        ("human", "{input}"), 
        ("placeholder", "{agent_scratchpad}"), 
    ]) 

    # Create the agent, binding the LLM, tools, and prompt together. 
    agent = create_tool_calling_agent(llm, tools, agent_prompt) 

    # AgentExecutor is the runtime that invokes the agent and executes the chosen tools. 
    # The 'tools' argument is not needed here as they are already bound to the agent. 
    agent_executor = AgentExecutor(agent=agent, verbose=True, 
    tools=tools) 
 
async def run_agent_with_tool(query: str): 
    """Invokes the agent executor with a query and prints the final 
    response.""" 
    print(f"\n--- 🏃 Running Agent with Query: '{query}' ---") 
    try: 
        response = await agent_executor.ainvoke({"input": query}) 
        print("\n--- ✅ Final Agent Response ---") 
        print(response["output"]) 
    except Exception as e: 
        print(f"\n🛑 An error occurred during agent execution: {e}") 
 
async def main(): 
    """Runs all agent queries concurrently.""" 
    tasks = [ 
        run_agent_with_tool("What is the capital of France?"), 
        run_agent_with_tool("What's the weather like in London?"),
        run_agent_with_tool("what is the tallest mountain on earth"),
        run_agent_with_tool("Tell me something about dogs.") # Should trigger the default tool response 
    ]
    await asyncio.gather(*tasks) 

    nest_asyncio.apply() 

if __name__ == "__main__":
    await main() 

Language model initialized: anthropic/claude-sonnet-4

--- 🏃 Running Agent with Query: 'What is the capital of France?' ---

--- 🏃 Running Agent with Query: 'What's the weather like in London?' ---

--- 🏃 Running Agent with Query: 'what is the tallest mountain on earth' ---

--- 🏃 Running Agent with Query: 'Tell me something about dogs.' ---


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Invoking: `search_information` with `{'query': 'weather in London'}`
responded: I'll help you find the current weather information for London.


--- 🛠 Tool Called: search_information with query: 'weather in London' ---
--- TOOL RESULT: The weather in London is currently cloudy with a temperature of 15°C. ---
The weather in London is currently cloudy with a temperature of 15°C.
Invoking: `search_information` with `{'query': 'capital of France'}`



--- 🛠 Tool Called: search_information with 